</center></div>
<div style = "background-color:indigo"><center>
<h1 style="font-size: 50px; font-weight: bold; color:goldenrod; border-top: 3px solid goldenrod; padding-top: 10px">OCEA California Legislative Policy Analysis (CALPA-LC)</h1>
<div style="font-size: 35px; font-weight: bold; color: goldenrod"> Part 1 - Preliminary Data Processing</div>
<div style="font-size: 30px; font-weight: bold; color: goldenrod; border-bottom: 3px solid goldenrod; padding-bottom: 20px">v.1.1 April 2025</div>
</center></div>

This is the main notebook for the AI California Legislative Policy Analysis (CALPA) project. The goal of this project is to analyze California legislative bills using natural language processing (NLP) techniques. This notebook will cover the preliminary data processing steps, including data loading, cleaning, and preparation for analysis.
The project is divided into several parts, each focusing on a specific aspect of the analysis. The first part will cover the data loading and cleaning process, while subsequent parts will focus on feature extraction, model training, and evaluation.

<h1 style="font-weight:bold; color:orangered; border-bottom: 2px solid orangered">1. Preliminaries</h1>

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">1.1 Referencing Libraries and Initialization</h2>

If needed to reset the kernel, please run the following cell:

In [ ]:
#%reset

Instantiating python libraries for the project

In [1]:
# Import required libraries
import os
from dotenv import load_dotenv
import time
from datetime import date
from datetime import datetime
import json
import mimetypes
import glob
import base64
import zipfile
import io
import requests
import pandas as pd

Load the local python modules containing classes and functions for the project from the local directory.
- `calpa`: This module contains the main classes and functions for the project, including the `LegiScan` class for the LegScan API.

In [2]:
# Load the calpa module located in the scripts/python/calpa directory
from calpa import calpa

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">1.2. Project and Workspace Variables</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Load Environment Variables</h3>

Define and maintain project, workspace and metadata. Below we load the environment variables from the `.env` file. The environment variables are used to configure the project and workspace settings. The `dotenv` library is used to load the environment variables from the `.env` file into the Python environment. The environment also contains the LegiScan API key, which is used to access the LegiScan API. The API key is stored in the `LEGISCAN_API_KEY` environment variable. The `dotenv` library is used to load the environment variables from the `.env` file into the Python environment.

In [3]:
# Load environment variables from .env file
load_dotenv()

True

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Main Class Instantiation</h3>

Instantiate the main class for the project:
- `legiscan`: This class is used to access the LegiScan API and retrieve legislative data.

In [4]:
# Instantiate the LegiScan class
legiscan = calpa.LegiScan()

Create project metadata for the project

In [5]:
# Create project metadata for the AI project
prjMetadata = calpa.projectMetadata(prjPart=1, prjComponent="LC", prjVersion="1.1")

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
 OCEA Legislative Policy Analysis (CaLPA-LC)
 California Legislative Policy Analysis for OCEA Legislative Committee Related Bills
 Part 1 - Preliminary Operations
 Version 1.1 (MIT License), Dr. Kostas Alexandridis, GISP
 GitHub Repository: https://github.com/ktalexan/CaLPA
 Last Updated: Apr 29, 2025
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Dates: 2024-12-02 through 2025-04-29
Periods: 2025-2026


Create the project directories dictionary

In [6]:
# Create the project directories dictionary
prjDirs = calpa.projectDirectories(os.getcwd())

Directory Global Settings:

General:
- Project (pathPrj): c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA
- Admin (pathAdmin): c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\admin
- Metadata (pathMetadata): c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\metadata
- Analysis (pathAnalysis): c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\analysis
- Obsidian Vault (pathObsidian): C:\Users\ktale\Knowledge Management\Policy and Governance\Legislation
Scripts:
- Python Calpa Module (pathScriptsCalpa): c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\calpa
- Markdown Scripts (pathScriptsMd): c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\markdown
- RIS Scripts (pathScriptsRis): c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\ris
Data:
- Main Data (pathData): c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\data
- Documents (pathDataDocs): c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\data\docs
- LegiScan (pathDataLegis): c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\data\legis
- LookUp (pathDataLo

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Lookup Data and Variables</h3>

If needed, you can access the project lookup and dictionary variables. They are embedded in the `codebook` module of the `calpa` package. The codebook module contains the following variables:
1. **LegiScan API Call Dictionaries**: These are codebook dictionaries that map the definitions of the LegiScan API fields for a number of API calls. The dictionaries available are (alphabetically ordered):
   - `dictGetAmendment`: Contains all the fields returned by the LegiScan API for the `getAmendment` call.
   - `dictGetBill`: Contains all the fields returned by the LegiScan API for the `getBill` call.
   - `dictGetBillText`: Contains all the fields returned by the LegiScan API for the `getBillText` call.
   - `dictGetPerson`: Contains all the fields returned by the LegiScan API for the `getPerson` call.
   - `dictGetRollCall`: Contains all the fields returned by the LegiScan API for the `getRollCall` call.
   - `dictGetSessionList`: Contains all the fields returned by the LegiScan API for the `getSessionList` call.
   - `dictGetSupplement`: Contains all the fields returned by the LegiScan API for the `getSupplement` call.
2. **Lookup Variables**: These are lists containing the names or definitions of codes used inside the LegiScan API calls (usually IDs or codes). The lookup variables available are (alphabetically ordered):
   - `lookupBillCode`: Bill code definitions used in the LegiScan API.
   - `lookupBillTextType`: IDs and definitions of the bill types used in the LegiScan API.
   - `lookupBillType`: IDs and definitions of the bill types used in the LegiScan API.
   - `lookupBodyType`: Definitions of body types used in the LegiScan API.
   - `lookupEventType`: Definitions of event types used in the LegiScan API.
   - `lookupMimeType`: Definitions of mime types used in the LegiScan API.
   - `lookupPartyType`: Definitions of party types used in the LegiScan API.
   - `lookupProgressType`: Definitions of progress types used in the LegiScan API.
   - `lookupReasonType`: Definitions of reason types used in the LegiScan API.
   - `lookupRoleType`: Definitions of role types used in the LegiScan API.
   - `lookupSastType`: Definitions of SAST types used in the LegiScan API.
   - `lookupSponsorType`: Definitions of sponsor types used in the LegiScan API.
   - `lookupStateType`: Definitions of state types used in the LegiScan API (California only).
   - `lookupStatusType`: Definitions of status types used in the LegiScan API.
   - `lookupSupplementType`: Definitions of supplement types used in the LegiScan API.
   - `lookupVoteType`: Definitions of vote types used in the LegiScan API.

If you need to access the codebook variables, you can do so by running the following cell. The codebook variables are stored in the `codebook` module of the `calpa` package. Calling the variables (assuming that the `calpa` package is imported) will return the variables as a dictionary. You can access the variables by using the dictionary keys. For example, to access the `lookupBillCode` variable, you can use the following code:

>```python
># Load the calpa module located in the scripts/python/calpa directory
>from calpa import calpa
>
># Getting the lookup variables from the codebook module directly
>calpa.codebook.lookupBillCode
>
># Assigning the lookup variables to a stored in the session
>lookupBillCode = calpa.codebook.lookupBillCode
>```


In [7]:
# Codebook lookup variables
codebookLookupVars = [var for var in dir(calpa.codebook) if var.startswith('lookup')]
codebookDictVars = [var for var in dir(calpa.codebook) if var.startswith('dict')]
print(f"Codebook Lookup Variables:\n{codebookLookupVars}\n")
print(f"Codebook Dictionary Variables:\n{codebookDictVars}\n")

Codebook Lookup Variables:
['lookupBillCode', 'lookupBillTextType', 'lookupBillType', 'lookupBodyType', 'lookupEventType', 'lookupMimeType', 'lookupPartyType', 'lookupProgressType', 'lookupReasonType', 'lookupRoleType', 'lookupSastType', 'lookupSponsorType', 'lookupStateType', 'lookupStatusType', 'lookupSupplementType', 'lookupVoteType']

Codebook Dictionary Variables:
['dictGetAmendment', 'dictGetBill', 'dictGetBillText', 'dictGetPerson', 'dictGetRollCall', 'dictGetSessionList', 'dictGetSupplement']



<h1 style="font-weight:bold; color:orangered; border-bottom: 2px solid orangered">2. Baseline LegiScan Data</h1>

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">2.1. Session List</h2>

Using the LegiScan API, we will retrieve the list of sessions for California. This will be used to get the session ID for the current session and the previous session. The session ID is needed to retrieve the bills for each session.

In [8]:
# Get the list of sessions from LegiScan
sessionList = legiscan.getSessionList()

Convert the session list to a pandas dataframe

In [9]:
# Convert the sessionList to a pandas DataFrame
sessionDf = pd.DataFrame(sessionList)
sessionDf.head()

,2009-2010,2011-2012,2013-2014,2015-2016,2017-2018,2019-2020,2021-2022,2023-2024,2025-2026
session_id,30,82,993,1120,1400,1624,1791,2016,2172
state_id,5,5,5,5,5,5,5,5,5
state_abbr,CA,CA,CA,CA,CA,CA,CA,CA,CA
year_start,2009,2011,2013,2015,2017,2019,2021,2023,2025
year_end,2010,2012,2014,2016,2018,2020,2022,2024,2026


We need to compare the session list we obtained from the Legiscan API with the previous session list (stored in the disk under `data/lookup/sessionList.json`). Here, we open the stored session list into a new dictionary called `sessionListStored`.

In [10]:
# Obtain the stored sessions list from JSON dictionary on disk (data/lookup directory)
sessionListStored = legiscan.getStoredData(dataType = "session")

Now that we have both the session lists (the one from the legiscan api, `sessionList`, and the stored version, `sessionListStored`), we can compare them. We will check if the session list from the LegiScan API is the same as the session list stored in the disk. If they are not the same, we will first identify which sessions need updating, and will later update the stored session list with the new session list from the LegiScan API. We will also check if there are any new sessions that have been added to the LegiScan API since the last time we retrieved the session list.

The function method `matchHash` from the legiscan module class, uses the hash values to compare the two lists. In this case the relevant JSON keys are `sesion_hash` for each `session_id`.

In [11]:
# Compare the sessionList and sessionListStored dictionaries for any changes
unmatchedSessions = legiscan.matchHash(sessionList, sessionListStored, hashType = "session", silent = True)

# if the unmatchedSessions is empty, print "All sessions match", and delete the unmatchedSessions variable
if unmatchedSessions is None:
    print("All sessions match")
    del unmatchedSessions
else:
    print("Unmatched sessions found")
    # Print the unmatched sessions
    print(unmatchedSessions)

All sessions match


Export the LegiScan query records to the `data/legis/json` directory as a JSON file for future reference.

In [ ]:
# Export the sessionList to a JSON file in the data/legiscan/json directory
with open(os.path.join(prjDirs["pathDataLegis"], "json", "sessionList.json"), "w", encoding="utf-8") as f:
    json.dump(sessionList, f, ensure_ascii=False, indent=4)
del f

If needed update the stored session list with the new session list from the LegiScan API.

In [ ]:
# Update the stored sessions list with the new sessionList
with open(os.path.join(prjDirs["pathDataLookup"], "sessionListStored.json"), "w", encoding="utf-8") as f:
    json.dump(sessionList, f, ensure_ascii=False, indent=4)
del f

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">2.2. Session People</h2>

In this step, we will obtain the list of California legislature members (Senate and Assembly) for each of the legislative sessions. This will be used to get the list of members for each session. The session ID is needed to retrieve the members for each session. We will use the LegiScan API to retrieve the list of members for each session.

The `legiscan.getSessionPeople` method retrieves the list of members for each session. The session ID is passed as an argument to the method. The method returns a list of members for each session. The list of members is stored in a dictionary called `sessionPeople`. The dictionary contains the session ID as the key and the list of members as the value.

In [12]:
# Get the list of session people from LegiScan
sessionPeople = {}
for key, value in sessionList.items():
    sessionId = value["session_id"]
    sessionPeople[key] = legiscan.getSessionPeople(sessionId)
del key, value, sessionId

Similarly with the legislative session list, we will compare the session people list we obtained from the LegiScan API with the previous session people list (stored in the disk under `data/lookup/sessionPeople.json`). Here, we open the stored session people list into a new dictionary called `sessionPeopleStored`.

In [13]:
# Obtain the stored session People list from JSON dictionary on disk (data/lookup directory)
sessionPeopleStored = legiscan.getStoredData(dataType = "people")

This time, the task is not that simple, since `sessionPeople` lists are nested for each session. The comparison of the Legislature members needs to be done in a loop for each session. In the following code segment, we perform this task in sequential steps:

1. Create a dictionary named `unmatchedPeople` to hold the unmatched session people (will be nested for each session).
2. Loop through the `sessionPeople` and `sessionPeopleStored` dictionaries to compare the session people lists for each session.
3. For each session, compare the session people lists and store the unmatched session people in the `unmatchedSessionPeople` dictionary, based on the `person_hash` key attribute on both lists. We will use the `matchHash` method from the `legiscan` module to compare the two lists.
4. If there are any unmatched session people, we will update the `unmatchedPeople` dictionary with the unmatched session people (for each session).
5. Finally, we will check if there are any unmatched session people in the `unmatchedPeople` dictionary. If there are, we will update the `sessionPeopleStored` dictionary with the unmatched session people and save it to the disk. 

In [14]:
# Compare the sessionPeople and sessionPeopleStored dictionaries for any changes
# Create a dictionary to store unmatched people
unmatchedPeople = {}
# Iterate through each session and compare the people lists
for key, value in sessionPeople.items():
    unmatchedPeople[key] = {}
    unmatched = legiscan.matchHash(sessionPeople[key]["people"], sessionPeopleStored[key]["people"], hashType = "person", silent = True)
    # If there are unmatched people, store them in the unmatchedPeople dictionary
    unmatchedPeople[key] = unmatched if unmatched is not None else None
del key, value, unmatched

# if the unmatchedPeople is empty, print "All people match", and delete the unmatchedPeople variable
if all(not value for value in unmatchedPeople.values()):
    print("All people match")
    # Delete the unmatchedPeople variable
    del unmatchedPeople
else:
    print("Unmatched people found")
    # Print the unmatched sessions
    print(unmatchedPeople)

All people match


Export the LegiScan query data for the session people to the `data/legis/json' directory for future reference.

In [ ]:
# Export the sessionPeople to a JSON file in the data/legiscan/json directory
with open(os.path.join(prjDirs["pathDataLegis"], "json", "sessionPeople.json"), "w", encoding="utf-8") as f:
    json.dump(sessionPeople, f, ensure_ascii=False, indent=4)
del f

If needed update the stored session people list with the new session people list from the LegiScan API.

In [ ]:
# Update the stored session People list with the new sessionPeople
with open(os.path.join(prjDirs["pathDataLookup"], "sessionPeopleStored.json"), "w", encoding="utf-8") as f:
    json.dump(sessionPeople, f, ensure_ascii=False, indent=4)
del f

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">2.3. Dataset List</h2>

In this session we will obtain the list of datasets and their attributes for each of the California Legislative Session from LegiScan. This process is needed to obtain the dataset `access_key` for each session, and consequently to use it in query dataset data in a further step.

The `legiscan.getDatasetList` method retrieves the list of datasets for each session. The session ID is passed as an argument to the method. The method returns a list of datasets for each session. The list of datasets is stored in a dictionary called `datasetList`. The dictionary contains the session ID as the key and the list of datasets as the value.

In [15]:
# Get the list of datasets from LegiScan for each legislative session
datasetList = legiscan.getDatasetList()

Obtain the stored dataset list from the disk. The list will be used to compare with the LegiScan API dataset list. The stored dataset list is stored in the `data/lookup/datasetList.json` file. The dataset list contains the list of datasets for each session. The dataset ID is needed to retrieve the datasets for each session. We will use the LegiScan API to retrieve the list of datasets for each session. Here, we open the stored dataset list into a new dictionary called `datasetListStored`.

In [16]:
# Obtain the stored dataset list from JSON dictionary on disk (data/lookup directory)
datasetListStored = legiscan.getStoredData(dataType = "dataset")

Now that we have both the dataset lists (the one from the legiscan api, `datasetList`, and the stored version, `datasetListStored`), we can compare them. We will check if the dataset list from the LegiScan API is the same as the dataset list stored in the disk. If they are not the same, we will first identify which datasets need updating, and will later update the stored dataset list with the new dataset list from the LegiScan API. We will also check if there are any new datasets that have been added to the LegiScan API since the last time we retrieved the session list.

The function method `matchHash` from the legiscan module class, uses the hash values to compare the two lists. In this case the relevant JSON keys are `dataset_hash` for each `session_id`.

In [17]:
# Compare the datasetList and datasetListStored dictionaries for any changes
unmatchedDatasets = legiscan.matchHash(datasetList, datasetListStored, hashType = "dataset", silent = True)

# if the unmatchedSessions is empty, print "All sessions match", and delete the unmatchedSessions variable
if unmatchedDatasets is None:
    print("All datasets match")
    del unmatchedDatasets
else:
    print("Unmatched datasets found")
    # Print the unmatched sessions
    print(unmatchedDatasets)

All datasets match


Export the LegiScan query records to the `data/legis/json` directory as a JSON file for future reference.

In [ ]:
# export the datasetList to a JSON file in the data/legis/json directory
with open(os.path.join(prjDirs["pathDataLegis"], "json", "datasetList.json"), "w", encoding="utf-8") as f:
    json.dump(datasetList, f, ensure_ascii=False, indent=4)
del f

Update the stored dataset list with the new dataset list from the LegiScan API.

In [ ]:
# Update the stored dataset list with the new datasetList
with open(os.path.join(prjDirs["pathDataLookup"], "datasetListStored.json"), "w", encoding="utf-8") as f:
    json.dump(datasetList, f, ensure_ascii=False, indent=4)
del f

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">2.4. Master List</h2>

This step is to obtain the master list of datasets for each session. The master list contains the list of bills for each legislative session. The master list is used to get the list of bills for each session. The session ID is needed to retrieve the bills for each session. We will use the LegiScan API to retrieve the list of bills for each session.

There are two options for this method. The first obtains the master list with bill attributes (when `raw = False`), and the second obtains the raw master list containing only the bill_ID and hash (when `raw = True`)

We will use the `legiscan.getMasterList(sessionID, raw)` method and will store the results in a dictionary called `masterList` or `masterListRaw` depending on the option provided in the method invocation.

In [18]:
# Get the Raw Master List from LegiScan for each legislative session
masterListRaw = {}
for key, value in sessionList.items():
    sessionId = value["session_id"]
    masterListRaw[key] = legiscan.getMasterList(sessionId, raw = True)
del key, value, sessionId

In [19]:
# Get the Master List from LegiScan for each legislative session
masterList = {}
for key, value in sessionList.items():
    sessionId = value["session_id"]
    masterList[key] = legiscan.getMasterList(sessionId, raw = False)
del key, value, sessionId

Obtain the stored master lists (both the raw and full) from the disk. The lists will be used to compare each bill with the LegiScan API master list. The stored dataset list is stored in the `data/lookup/mastertList.json` or `data/lookup/masterListRaw.json` files. The master lists contain the list of bills for each session. The dataset ID is needed to retrieve the bills for each session. We will use the LegiScan API to retrieve the list of datasets for each session. Here, we open the stored dataset list into a new dictionary called `masterListStored` and `masterListRawStored`.

In [20]:
# Get the stored raw master list from JSON dictionary on disk (data/lookup directory)
masterListRawStored = legiscan.getStoredData(dataType = "master", raw = True)
# Get the stored master list from JSON dictionary on disk (data/lookup directory)
masterListStored = legiscan.getStoredData(dataType = "master", raw = False)

Now that we have both the master lists (the one from the legiscan api, `masterListRaw`, and the stored version, `masterListRawStored`), we can compare them. We will check if the master list from the LegiScan API is the same as the master list stored in the disk. If they are not the same, we will first identify which bills need updating, and will later update the stored master list with the new master list from the LegiScan API. We will also check if there are any new bills that have been added to the LegiScan API since the last time we retrieved the session list.

The function method `matchHash` from the legiscan module class, uses the hash values to compare the two lists. In this case the relevant JSON keys are `chanbe_hash` for each `session_id`.

In [21]:
# Compare the masterList and masterListStored dictionaries for any changes
# Create a dictionary to store unmatched bills
unmatchedMasterList = {}
# Iterate through each session and compare the bills lists
for key, value in masterList.items():
    unmatchedMasterList[key] = {}
    unmatched = legiscan.matchHash(masterList[key]["bills"], masterListStored[key]["bills"], hashType = "change", silent = True)
    # If there are unmatched bills, store them in the unmatchedMasterList dictionary
    unmatchedMasterList[key] = unmatched if unmatched is not None else None
del key, value, unmatched

if all(not value for value in unmatchedMasterList.values()):
    print("All master lists match")
    # Delete the unmatchedMasterList variable
    del unmatchedMasterList
else:
    print("Unmatched master lists found")
    # Print the unmatched sessions
    print(unmatchedMasterList)

All master lists match


In [22]:
# Compare the masterList and masterListStored dictionaries for any changes
# Create a dictionary to store unmatched bills
unmatchedMasterListRaw = {}
# Iterate through each session and compare the bills lists
for key, value in masterListRaw.items():
    unmatchedMasterListRaw[key] = {}
    unmatched = legiscan.matchHash(masterListRaw[key]["bills"], masterListRawStored[key]["bills"], hashType = "change", silent = True)
    # If there are unmatched bills, store them in the unmatchedMasterList dictionary
    unmatchedMasterListRaw[key] = unmatched if unmatched is not None else None
del key, value, unmatched

if all(not value for value in unmatchedMasterListRaw.values()):
    print("All master lists match")
    # Delete the unmatchedMasterList variable
    del unmatchedMasterListRaw
else:
    print("Unmatched master lists found")
    # Print the unmatched sessions
    print(unmatchedMasterListRaw)

All master lists match


Export both the LegiScan query records (raw and full master list) to the `data/legis/json` directory as a JSON file for future reference.

In [ ]:
# export the raw master list to a JSON file in the data/legis/json directory
with open(os.path.join(prjDirs["pathDataLegis"], "json", "masterListRaw.json"), "w", encoding="utf-8") as f:
    json.dump(masterListRaw, f, ensure_ascii=False, indent=4)
del f

In [ ]:
# export the master list to a JSON file in the data/legis/json directory
with open(os.path.join(prjDirs["pathDataLegis"], "json", "masterList.json"), "w", encoding="utf-8") as f:
    json.dump(masterList, f, ensure_ascii=False, indent=4)
del f

If needed update the stored session people list with the new session people list from the LegiScan API.

In [ ]:
# Update the stored raw master list with the new masterListRaw
with open(os.path.join(prjDirs["pathDataLookup"], "masterListRawStored.json"), "w", encoding="utf-8") as f:
    json.dump(masterListRaw, f, ensure_ascii=False, indent=4)
del f

In [ ]:
# Update the master list with the new masterList
with open(os.path.join(prjDirs["pathDataLookup"], "masterListStored.json"), "w", encoding="utf-8") as f:
    json.dump(masterList, f, ensure_ascii=False, indent=4)
del f

<h1 style="font-weight:bold; color:orangered; border-bottom: 2px solid orangered">3. Bill Monitoring Operations</h1>

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.1. Stored Monitored LC Bill List</h2>

Open the stored LC list of monitoring bills from disk (`data/lookup/lcBillListStored.json`). The list will be used to compare with the LegiScan API bill list.

In [39]:
# Get the list of LC bills from the stored list on disk (data/lookup directory)
lcBillListStored = legiscan.getStoredData(dataType = "bills", project = "LC")

Now that we have both the AI bill lists (the master list from the legiscan api, `masterListRaw`, and the stored version, `lcBillListStored`), we can compare them. We will check if the bills from the LC bill list stored in the disk are the same as the LegiScan API bills in the master list. If they are not the same, we will first identify which bills need updating, and will later update the stored LC bill list with the new bill list from the LegiScan API. We will also check if there are any new bills that have been added to the LegiScan API since the last time we retrieved the session list.

The function method `matchHash` from the legiscan module class, uses the hash values to compare the two lists. In this case the relevant JSON keys are `change_hash` for each `session_id`.

In [41]:
# Compare the lcBillListStored and the masterListRaw dictionaries for any changes
unmatchedLcBillList = {}
# Iterate through each session and compare the bills lists
for key, value in lcBillListStored.items():
    unmatchedLcBillList[key] = {}
    unmatched = legiscan.matchHash(lcBillListStored[key], masterListRaw[key]["bills"], hashType = "change", silent = True)
    # If there are unmatched bills, store them in the unmatchedLcBillList dictionary
    unmatchedLcBillList[key] = unmatched if unmatched is not None else None
del key, value, unmatched

# if the unmatchedLcBillList is empty, print "All LC bills match", and delete the unmatchedLcBillList variable
if all(not value for value in unmatchedLcBillList.values()):
    print("All LC bills match")
    # Delete the unmatchedLcBillList variable
    del unmatchedLcBillList
else:
    print("Unmatched LC bills found")
    # Print the unmatched sessions
    print(unmatchedLcBillList)

All LC bills match


If needed, update the stored LegiScan AI search query list with the new LegiScan AI search query list from the LegiScan API.

In [42]:
# Update the stored aiBillList with the new aiBillList
with open(os.path.join(prjDirs["pathDataLookup"], "lcBillListStored.json"), "w", encoding="utf-8") as f:
    json.dump(lcBillListStored, f, ensure_ascii=False, indent=4)
del f

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.3. Get LegiScan Bills</h2>

Get the LegiScan bills for each session. The session ID is needed to retrieve the bills for each session. We will use the LegiScan API to retrieve the list of bills for each session.

In [43]:
# Define a dictionary to store LC bills
lcBills = {}
j = 0
# Iterate through the AI bill list and fetch the bill details from LegiScan
for myPeriod, myBillList in lcBillListStored.items():
    # Set the key to the legislative session period
    lcBills[myPeriod] = {}
    i = 1
    l = len(myBillList)
    print(f"\n{myPeriod} Legislative Session ({l} bills)")
    # Iterate through the bills for the current legislative session
    for myBillNumber, myBill in myBillList.items():
        # Get the bill ID
        myBillId = myBill["bill_id"]
        print(f"  {i}. {myBillNumber} ({myBillId})")
        # add the legiscan query to the aiBills dictionary
        lcBills[myPeriod][myBillNumber] = legiscan.getBill(billId = myBillId)
        i += 1
    j += i - 1
print(f"\nCompleted fetching AI bills from LegiScan (Total: {j} bills)")
del myPeriod, myBillList, myBillNumber, myBillId, myBill, i, l, j


2025-2026 Legislative Session (58 bills)
  1. AB21 (1893357)
  2. AB23 (1893359)
  3. AB33 (1893370)
  4. AB96 (1904796)
  5. AB231 (1916611)
  6. AB283 (1936624)
  7. AB288 (1936629)
  8. AB303 (1939179)
  9. AB331 (1943413)
  10. AB339 (1945201)
  11. AB340 (1945202)
  12. AB346 (1947437)
  13. AB423 (1958468)
  14. AB424 (1958469)
  15. AB427 (1958472)
  16. AB478 (1964665)
  17. AB495 (1964682)
  18. AB522 (1964709)
  19. AB569 (1968210)
  20. AB571 (1968212)
  21. AB596 (1970384)
  22. AB672 (1971702)
  23. AB711 (1971741)
  24. AB853 (1976440)
  25. AB882 (1976469)
  26. AB1054 (1978589)
  27. AB1067 (1978602)
  28. AB1109 (1978644)
  29. AB1189 (1980086)
  30. AB1323 (1980220)
  31. AB1331 (1980228)
  32. AB1355 (1980252)
  33. AB1383 (1980280)
  34. ACA1 (1893407)
  35. ACA2 (1893408)
  36. SB7 (1893452)
  37. SB16 (1893428)
  38. SB21 (1893434)
  39. SB23 (1893436)
  40. SB27 (1893440)
  41. SB28 (1893441)
  42. SB35 (1894445)
  43. SB36 (1894446)
  44. SB37 (1894447)
  45. S

Export the LegiScan query records to the `data/legis/json` directory as a JSON file for future reference.

In [44]:
# Export the AI bills to a JSON file in the data/legiscan/json directory
with open(os.path.join(prjDirs["pathDataLegis"], "json", "lcBills.json"), "w", encoding="utf-8") as f:
    json.dump(lcBills, f, ensure_ascii=False, indent=4)
del f

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.3. Get Bill Text</h2>

Get the LegiScan bill text for each session. This process involves two legiscan functions:

- `legiscan.getBillText`: This function retrieves the bill text for each bill in the session. The document ID is passed as an argument to the function. The function runs the LegiScan API call, and returns the bill text JSON information for each bill (which includes the base64 encoded bill text).
- `legiscan.summarizeBillText`: This function summarizes the bill text for each bill in the session. The bill JSON information is passed as an argument to this function. The function performs a number of tasks:
    - Looks up the `texts` JSON object group of the bill, and finds the last bill text version (is the last one in the list, with the latest bill date), and retrieves the `doc_id` identifier of the bill text. 
    - Uses the `legiscan.getBillText` function above to retrieve the encoded (base64) bill text. It then proceeds to decode the encoded bill text, and converts it to a string.
    - Uses an `Azure OpenAI` API call to create a TL;DR summary of the bill text, along with a list of keywords (tags) for the bill text. 
    - Finally, it constructs and returns a dictionary with the bill number, summarized bill text, the tags, and the bill text itself.
 

In [ ]:
# Create a function that loops through the aiBills dictionary, gets the bill text for each bill, and then processes the text through Azure OpenAI to obtain the summary and keywords.
def createBillTextSummary(billList):
    """
    Create a dictionary of bill text objects from the bill list.
    """
    # Create a dictionary to store the bill text objects
    billTextDict = {}
    # First, loop through the legislative sessions in the bill list
    for key, value in billList.items():
        # Set the key to the legislative session period
        billTextDict[key] = {}
        # Count the number of bills in the legislative session
        billCount = len(value)
        print(f"\n{key} Legislative Session ({billCount} bills)")
        # Reset the counter for the number of bills processed
        i = 1        
        # Loop through the bills for the legislative session
        for billKey, billContent in value.items():
            maxRetries = 5
            attempts = 0
            success = False
            while not success and attempts < maxRetries:
                try:
                    # Get the bill text from LegiScan
                    billTextJson = legiscan.summarizeBillText(billContent)
                    success = True
                except Exception as e:
                    attempts += 1
                    print(f"  - Error fetching bill text for {billKey}: {e}")
            if success:
                # If the bill text is processed,
                print(f"  {billKey} ({i}/{billCount})")
                # Add the bill text to the billTextDict dictionary
                billTextDict[key][billKey] = billTextJson
                # Increment the counter for the number of bills processed
                i += 1
            else:
                break
    # Return the billTextDict dictionary
    return billTextDict

We will execute the `createBillTextDict` function above (which in turns runs the `legiscan.summarizeBillText` function) for each bill in each legislative session, and generates a new dictionary obtaining the resulting data.

In [46]:
lcBillsSummaries = createBillTextSummary(lcBills)


2025-2026 Legislative Session (58 bills)
  AB21 (1/58)
  AB23 (2/58)
  AB33 (3/58)
  AB96 (4/58)
  AB231 (5/58)
  AB283 (6/58)
  AB288 (7/58)
  AB303 (8/58)
  AB331 (9/58)
  AB339 (10/58)
  AB340 (11/58)
  AB346 (12/58)
  AB423 (13/58)
  AB424 (14/58)
  AB427 (15/58)
  AB478 (16/58)
  AB495 (17/58)
  AB522 (18/58)
  AB569 (19/58)
  AB571 (20/58)
  AB596 (21/58)
  AB672 (22/58)
  AB711 (23/58)
  AB853 (24/58)
  AB882 (25/58)
  AB1054 (26/58)
  AB1067 (27/58)
  AB1109 (28/58)
  AB1189 (29/58)
  AB1323 (30/58)
  AB1331 (31/58)
  AB1355 (32/58)
  AB1383 (33/58)
  ACA1 (34/58)
  ACA2 (35/58)
  SB7 (36/58)
  SB16 (37/58)
  SB21 (38/58)
  SB23 (39/58)
  SB27 (40/58)
  SB28 (41/58)
  SB35 (42/58)
  SB36 (43/58)
  SB37 (44/58)
  SB70 (45/58)
  SB82 (46/58)
  SB83 (47/58)
  SB237 (48/58)
  SB238 (49/58)
  SB299 (50/58)
  SB301 (51/58)
  SB309 (52/58)
  SB366 (53/58)
  SB458 (54/58)
  SB481 (55/58)
  SB600 (56/58)
  SB853 (57/58)
  SJR2 (58/58)


Export the LegiScan query records to the `data/legis/json` directory as a JSON file for future reference.

In [47]:
# Export the AI bills summaries to a JSON file in the data/legiscan/json directory
with open(os.path.join(prjDirs["pathDataLegis"], "json", "lcBillsSummaries.json"), "w", encoding="utf-8") as f:
    json.dump(lcBillsSummaries, f, ensure_ascii=False, indent=4)
del f

In [48]:
# Export the AI bill summaries to a JSON file in the data/lookup directory for reference
with open(os.path.join(prjDirs["pathDataLookup"], "lcBillsSummariesStored.json"), "w", encoding="utf-8") as f:
    json.dump(lcBillsSummaries, f, ensure_ascii=False, indent=4)
del f

<div style = "background-color:indigo"><center>
<h1 style="font-weight:bold; color:goldenrod; border-top: 2px solid goldenrod; border-bottom: 2px solid goldenrod; padding-top: 5px; padding-bottom: 10px">End of Script</h1>
</center></div>